In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
import joblib
import pickle

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn_crfsuite.metrics import flat_classification_report

In [ ]:
data_path='./data/bind_mark.txt'        #mark 資料

# 輸入path給data list
def Dataset(data_path):
    with open(data_path, 'r', encoding='utf-8') as f:
        data=f.readlines()#.encode('utf-8').decode('utf-8-sig')
    data_list, data_list_tmp = list(), list()
    article_id_list=list()
    idx=0
    for row in data:
        data_tuple = tuple()
        if row == '\n':
            article_id_list.append(idx)
            idx+=1
            data_list.append(data_list_tmp)
            data_list_tmp = []
        else:
            row = row.strip('\n').split(' ')
            data_tuple = (row[0], row[1])
            data_list_tmp.append(data_tuple)
    if len(data_list_tmp) != 0:
        data_list.append(data_list_tmp)
    
    return data_list, article_id_list

# 輸入mark的 data_list 資料給出 Y 值
def Preprocess(data_list):
    label_list = list()
    for idx_list in range(len(data_list)):
        label_list_tmp = list()
        for idx_tuple in range(len(data_list[idx_list])):
            label_list_tmp.append(data_list[idx_list][idx_tuple][1])
        label_list.append(label_list_tmp)
    return label_list

#輸入為 一個 pickle 檔
def get_X(data):
    output = []
    for sen in data:
        for m in sen:
            for col in range(1, m.shape[1]-1):
                d = {}
                for row in range(m.shape[0]):
                    d['dim_{}'.format(str(row+1))] = m[row, col]
                output.append(d)
    return output

In [ ]:
# 獲得 y
data_list, article_id_list = Dataset(data_path)
y = Preprocess(data_list)

x_train = []
for i in range(1, 147):
    with open('./embedding_data/data{}.pickle'.format(str(i)), 'rb') as f:
        p = pickle.load(f)
    x_train.append(get_X(p))

In [ ]:
def CRF(X_train, y_train):
    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=True
    )
    crf.fit(X_train, y_train)
    
    return crf

crf = CRF(x_train, y)
#joblib.dump(crf, './data/model_crf_bert.joblib')

In [ ]:
x_test = []
for i in range(1, 41):
    with open('./embedding_data/dev/dev{}.pickle'.format(str(i)), 'rb') as f:
        p = pickle.load(f)
    x_test.append(get_X(p))

y_pred = crf.predict(x_test)

In [ ]:
with open('./data/develop1.pickle', 'rb') as f:
    test = pickle.load(f)

testdata = []
for i, j in test.items():
    s = ''
    for k in range(len(j[0])):
        s = s + j[0][k] + j[1][k]
    testdata.append(s)

test_id_list = [i for i in range(40)]

In [ ]:
output="article_id\tstart_position\tend_position\tentity_text\tentity_type\n"
for test_id in range(len(y_pred)):
    pos=0
    start_pos=None
    end_pos=None
    entity_text=None
    entity_type=None
    for pred_id in range(len(y_pred[test_id])):
        if pred_id+1 == len(y_pred[test_id]):
            c = 'O'
        else:
            c = y_pred[test_id][pred_id+1][0]
        
        if y_pred[test_id][pred_id][0]=='B':
            start_pos=pos
            entity_type=y_pred[test_id][pred_id][2:]
        elif start_pos is not None and y_pred[test_id][pred_id][0]=='I' and c=='O':
            end_pos=pos
            entity_text=''.join([testdata[test_id][position][0] for position in range(start_pos,end_pos+1)])
            line=str(test_id_list[test_id])+'\t'+str(start_pos)+'\t'+str(end_pos+1)+'\t'+entity_text+'\t'+entity_type
            output+=line+'\n'
        pos+=1
    
output_path='./data/output.csv'
with open(output_path,'w',encoding='utf-8') as f:
    f.write(output)
    
print(output)